In [1]:
%pip install -Uqqq pip --progress-bar off
%pip install -qqq ollama --progress-bar off
%pip install -qqq pathlib --progress-bar off
%pip install -qqq pandas --progress-bar off
%pip install -qqq PyPDF2 --progress-bar off
%pip install -qqq ollama --progress-bar off
%pip install -qqq owlready2 --progress-bar off
%pip install -qqq rdflib --progress-bar off


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
from enum import Enum
import PyPDF2
import ollama
from collections import defaultdict
import re
from datetime import datetime
import unicodedata

MODEL = "llama3.1:8b-instruct-q8_0"
 

In [3]:
def extract_pdf_text(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = " ".join(page.extract_text() for page in reader.pages)
    return text

pdf_text = extract_pdf_text("sensors.pdf")
print("text extracted from pdf")

text extracted from pdf


In [4]:
class ResponseFormat(Enum):
    JSON = "json_object"
    TEXT = "text"
 
 
def call_model(
    prompt: str, response_format: ResponseFormat = ResponseFormat.TEXT
) -> str:
    response = ollama.generate(
        model=MODEL,
        prompt=prompt,
        keep_alive="1h",
        format="" if response_format == ResponseFormat.TEXT else "json",
    )
    return response["response"]

In [41]:

SUMMARIZE_PROMPT = """
As a LiDAR sensor expert, your task is to extract, categorize, and group all the **real, specific, and branded** LiDAR sensor names mentioned in the following text.

**Output Requirements**:
1. Provide a concise and structured list of LiDAR sensors.
2. Group and standardize similar categories to avoid duplicates (e.g., "Automotive LiDAR", "Automotive LiDAR Sensors", and "Automotive Sensors" should be grouped under one category, "Automotive LiDAR").
3. Categorize the sensors based on their application, type, or any mentioned specifications (e.g., Automotive, Industrial, Surveying).
4. Include only **specific branded or model names** of sensors (e.g., "Velodyne Velarray", "Livox Horizon"). 

**Exclusion Criteria**:
- Do not include generic terms such as "LiDAR", "LiDAR sensor", "LiDAR sensors", "various LiDAR sensors", or "no specific brand model mentioned".
- Exclude datasets or evaluation tools (e.g., "Kitti", "NuScenes").
- Ignore descriptions or summaries like "solid-state LiDAR", "lidar technology", or "selected sensors are mechanical or solid-state types".
- Avoid placeholder entries like "not mentioned" or "various".

**Formatting**:
- Strictly follow the JSON template below:
  {{
    "sensors": [
      {{
        "name": "Lidar Sensor 1",
        "category": "Category Name",
      }}
    ]
  }}

**Additional Instructions**:
- Only include a category if it contains sensors.
- Use consistent naming conventions for categories. Avoid redundancy or variations in names.
- If no sensors are found in the text, return an empty JSON: {{}}.
- Do not include any additional explanations or information.

<text>
{text}
</text>

"""

In [42]:
# Step 1: Chunk size for processing
CHUNK_SIZE = 1000
chunks = [pdf_text[i:i + CHUNK_SIZE] for i in range(0, len(pdf_text), CHUNK_SIZE)]

# Save chunks to a file if needed
with open("intermediate_data/chunks.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f)

print("Chunks created and saved.")


Chunks created and saved.


In [43]:
# Step 2: Process chunks with the model and save responses
responses = []

for idx, chunk in enumerate(chunks):
    formatted_prompt = SUMMARIZE_PROMPT.format(text=chunk)
    response = call_model(formatted_prompt)  # Call your expensive model
    responses.append(response)

    # Save responses after every chunk to ensure progress is retained
    with open("intermediate_data/responses.json", "w", encoding="utf-8") as f:
        json.dump(responses, f)

print("Model responses saved.")


Model responses saved.


In [ ]:
import json
import unicodedata
import re

# Function to clean strings
def clean_string(s):
    s = s.strip()  # Remove leading/trailing whitespace
    s = s.encode('utf-8').decode('unicode_escape')  # Decode Unicode escapes
    s = unicodedata.normalize('NFKC', s)  # Normalize special characters
    s = re.sub(r"[\s\-_/]+", " ", s)  # Replace special separators with space
    s = s.replace("\u00c2", "")  # Remove unwanted artifacts
    return s.lower()

# Load responses and process them
with open("intermediate_data/responses.json", "r", encoding="utf-8") as f:
    responses = json.load(f)

# Placeholder for final results and a set to track unique sensors
final_sensors = []
seen_sensors = set()

# Process responses into final_sensors
for response in responses:
    try:
        response_json = json.loads(response)

        if "sensors" in response_json and isinstance(response_json["sensors"], list):
            for sensor in response_json["sensors"]:
                sensor_name = clean_string(sensor.get("name", "Unnamed Sensor"))
                category_name = clean_string(sensor.get("category", "Uncategorized"))

                # Create a tuple to represent the sensor as a unique combination of name and category
                sensor_tuple = (sensor_name, category_name)

                # If the combination is not seen, add it to the list
                if sensor_tuple not in seen_sensors:
                    seen_sensors.add(sensor_tuple)
                    final_sensors.append({
                        "name": sensor_name,
                        "category": category_name
                    })
    except json.JSONDecodeError as e:
        print(f"Error parsing response: {response}. Details: {e}")

# Save the cleaned and unique sensors to a new file
with open("final_sensors_no_duplicates.json", "w", encoding="utf-8") as out_file:
    json.dump({"sensors": final_sensors}, out_file, ensure_ascii=False, indent=4)

print("Sensors processed, duplicates removed, and saved to final_sensors_no_duplicates.json.")


In [57]:
from owlready2 import *

# Create a new ontology
onto = get_ontology("http://example.org/sensors.owl")

# Define classes (Category first, then Sensor as subclass)
with onto:
    class Category(Thing):
        pass

    class Sensor(Category):  # Sensor is a subclass of Category
        pass
    class has_sensor(ObjectProperty):
        domain = [Category]
        range = [Sensor]

# Add individual sensors and categories
for sensor in final_sensors:
    category_class = types.new_class(sensor["category"], (Category,))
    sensor_class = types.new_class(sensor["name"], (Sensor,))
    category_class.is_a.append(has_sensor.some(sensor_class))
# Save the ontology
onto.save(file="sensors_ontology.owl", format="rdfxml")
print("Ontology created and saved.")


Ontology created and saved.


In [10]:
from owlready2 import get_ontology, Thing, ObjectProperty
import re
import traceback

# Helper function to clean names
def clean_name(name):
    return re.sub(r'\W+', '_', name).strip('_')

def create_ontology_from_json(json_data, ontology_url="http://example.org/lidar.owl", output_file="lidar_ontology.owl"):
    ontology = get_ontology(ontology_url)

    with ontology:
        # Define core classes
        class LidarCategory(Thing):
            pass

        class Sensor(Thing):
            pass

        class has_sensor(ObjectProperty):
            domain = [LidarCategory]
            range = [Sensor]

        sensor_classes = {}

        for category in json_data.get("categories", []):
            category_name = clean_name(category["name"])
            
            try:
                # Create category class only once
                category_class = type(category_name, (LidarCategory,), {})
            except Exception as e:
                print(f"Error creating category class '{category_name}': {e}")
                traceback.print_exc()
                continue

            for sensor in category.get("sensors", []):
                sensor_name = clean_name(sensor)
                
                if sensor_name not in sensor_classes:
                    try:
                        # Create sensor class only once
                        sensor_class = type(sensor_name, (Sensor,), {})
                        sensor_classes[sensor_name] = sensor_class
                    except Exception as e:
                        print(f"Error creating sensor class '{sensor_name}': {e}")
                        traceback.print_exc()
                        continue
                else:
                    sensor_class = sensor_classes[sensor_name]

                try:
                    # Add restriction only once per sensor class
                    category_class.is_a.append(has_sensor.some(sensor_class))
                except Exception as e:
                    print(f"Error adding restriction for '{category_name}' -> '{sensor_name}': {e}")
                    traceback.print_exc()
                    continue

    try:
        ontology.save(file=output_file, format="rdfxml")
        print(f"OWL file created at: {output_file}")
    except Exception as e:
        print(f"Error saving OWL file: {e}")
        traceback.print_exc()
json_data = json.loads(final_output)
create_ontology_from_json(json_data)

OWL file created at: lidar_ontology.owl


In [11]:
json_data = json.loads(final_output)

# Define output OWL file path
output_owl_file = f"out/OWL/LIDAR{datetime.now().strftime('%d.%m.%Y_%H%M%S')}.owl"
# Generate the OWL ontology
create_ontology_from_json(json_data, output_file=output_owl_file)

OWL file created at: out/OWL/LIDAR06.12.2024_123654.owl


In [12]:
OWL_PROMPT = """
You are an ontology engineering expert specializing in converting structured JSON data into Web Ontology Language (OWL) representations.

**Task:** Transform the provided JSON data into a fully functional OWL ontology that captures all semantic relationships, hierarchical structures, and details present in the data.

**Input JSON Structure:**

{JSON}

### Ontology Creation Guidelines:

1. **Analyze the JSON Structure**:
   - Identify all entities: categories, sensors, and their relationships.
   - Treat each category as a subclass of the "Category" superclass.
   - Treat each sensor as a subclass of the "Sensor" superclass.
   - Explicitly represent every sensor listed in the JSON as a subclass of `Sensor`.

2. **Create OWL Classes**:
   - Use "Thing" as the base class for all entities.
   - Create "Category" and "Sensor" as direct subclasses of "Thing".
   - Define each category['name'] from the JSON as a subclass of "Category".
   - Define each element of the array category['sensor'] from the JSON as a subclass of "Sensor".
   - Ensure every sensor listed under a category['sensor'] is included as a class.

3. **Define Relationships**:
   - Use object properties to capture relationships:
     - **belongsToCategory**: Links each sensor to the category it belongs to.
     - **hasSensor**: Links each category to the sensors it contains.
   - For each sensor:
     - Create a restriction using **someValuesFrom** to link it to its category via the `belongsToCategory` property.
   - For each category:
     - Create an object property linking it to its sensors using the `hasSensor` property.

4. **Handle Categories Without Sensors**:
   - If a category has no sensors listed, create the category class and add a comment (`rdfs:comment`) indicating this.

5. **Ontological Structure**:
   - Ensure a hierarchical structure:
     - **Thing** as the root class.
     - **Category** and **Sensor** as direct subclasses.
   - Represent categories and sensors as subclasses under their respective hierarchies.
   - Use meaningful naming conventions (e.g., PascalCase for classes, camelCase for properties).
   - Include comments (`rdfs:comment`) for every class, property, and relationship.

6. **Validation and Completeness**:
   - Represent all categories and sensors, including categories with no sensors or overlapping names.
   - Explicitly capture all semantic relationships for clarity and completeness.
   - Validate the ontology for logical consistency.

7. **Example Ontology Structure**:
   - Classes:
     - Base: Thing
     - Subclasses: Category and Sensor
     - Derived classes for each category and sensor (e.g., AutomotiveLidar, BlickfeldCubeSensors2022).
   - Object Properties:
     - **belongsToCategory**: Links sensors to their categories.
     - **hasSensor**: Links categories to their sensors.
   - Relationships:
     - If "Automotive Lidar" contains "Blickfeld Cube Sensors 2022", create:
       ```xml
       <owl:Class rdf:about="#BlickfeldCubeSensors2022">
           <rdfs:label>Blickfeld Cube Sensors 2022</rdfs:label>
           <rdfs:subClassOf rdf:resource="#Sensor"/>
           <rdfs:subClassOf>
               <owl:Restriction>
                   <owl:onProperty rdf:resource="#belongsToCategory"/>
                   <owl:someValuesFrom rdf:resource="#AutomotiveLidar"/>
               </owl:Restriction>
           </rdfs:subClassOf>
       </owl:Class>
       ```
       ```xml
       <owl:Class rdf:about="#AutomotiveLidar">
           <rdfs:label>Automotive Lidar</rdfs:label>
           <rdfs:subClassOf rdf:resource="#Category"/>
           <hasSensor rdf:resource="#BlickfeldCubeSensors2022"/>
       </owl:Class>
       ```

   - For empty categories, add a comment:
     ```xml
     <owl:Class rdf:about="#NoCategory">
         <rdfs:label>No Category</rdfs:label>
         <rdfs:comment>No sensors were listed for this category in the JSON data.</rdfs:comment>
     </owl:Class>
     ```

**Ontology Creation Guidelines:**
- Use valid OWL and RDF/XML syntax suitable for tools like Protégé.

**Output Requirements:**
1. Provide **only the complete OWL ontology in RDF/XML format**.
2. Do not include explanations, code snippets, or Python scripts.
3. The output must start with `<?xml version="1.0"?>` and end with `</rdf:RDF>`.
4. Exclude any additional comments, explanations, or programming instructions.
**Generate the complete OWL ontology based on these guidelines.**

"""

OWL_RESPONSE = call_model(OWL_PROMPT.format(JSON=final_output));

In [13]:
#save the owl file
with open("Extracted_Data2.owl", "w", encoding="utf-8") as f:
    f.write(OWL_RESPONSE)